# Análisis Exploratorio de Datos: Accidentes de Tráfico en España (2023)

Este notebook presenta un análisis exploratorio completo de los accidentes de tráfico en España durante 2023, combinando datos de accidentes con información del parque de vehículos por provincia.

## Estructura del Análisis

### 1. Análisis Exploratorio Inicial
1.1. Visión General del Dataset
- Dimensiones y estructura
- Tipos de datos
- Análisis de valores nulos
- Resumen estadístico básico

1.2. Distribución Temporal
- Patrones mensuales
- Distribución semanal
- Análisis por tramos horarios
- Períodos críticos

1.3. Distribución Geográfica
- Análisis provincial
- Tipos de zona
- Análisis por tipo de vía
- Mapas de accidentalidad

### 2. Análisis de Severidad y Víctimas por Tipo de Usuario
- Distribución por tipo
- Comparativas de severidad
- Patrones específicos

### 3. Factores Contextuales
3.1. Condiciones Ambientales
- Iluminación
- Condiciones meteorológicas
- Estado del firme
- Visibilidad

3.2. Características de la Vía
- Titularidad
- Tipos de vía
- Zonas de concentración

### 4. Análisis de Riesgo por Tipo de Vehículo

4.1 Relación entre Composición del Parque y Accidentalidad
- Analizar la composición del parque de vehículos en España
- Comparar con la implicación en accidentes mortales de cada tipo
- Identificar vehículos con riesgo desproporcionado
- Calcular índices de riesgo relativo por categoría de vehículo

4.2 Análisis Complementario: Composición del Parque Vehicular por Provincia
- Distribución de los diferentes tipos de vehículos en las provincias de España.
- Relación con muertes

### 5. Conclusiones
- Informe del Análisis realizado
- Hallazgos principales
- Factores de riesgo
- Recomendaciones

In [1]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import scipy.stats as stats
import folium
from IPython.display import display, HTML

# Configuración de visualización
sns.set_theme()
plt.rcParams['figure.figsize'] = (12, 6)

## 1. Análisis Exploratorio Inicial

### 1.1 Visión General del Dataset

Comenzamos cargando el dataset enriquecido que contiene tanto la información de accidentes como la del parque de vehículos, y realizaremos un análisis inicial de su estructura y contenido.

In [2]:
# Definir paths y cargar datos
PROC = Path('../data/processed')
df = pd.read_csv(PROC / 'accidentes_2023_enriquecido.csv')

# Información básica del dataset
print("Dimensiones del dataset:", df.shape)
print("\nColumnas disponibles:")
print(df.columns.tolist())

# Información detallada de tipos de datos y valores no nulos
print("\nInformación detallada del dataset:")
df.info()

Dimensiones del dataset: (101306, 43)

Columnas disponibles:
['anio', 'mes', 'dia_semana', 'hora', 'cod_provincia', 'cod_municipio', 'mes_label', 'dia_semana_label', 'provincia_label', 'tramo_horario_label', 'zona_label', 'zona_agrupada_label', 'tipo_via_label', 'sentido_label', 'titularidad_via_label', 'tipo_accidente_label', 'condicion_iluminacion_label', 'condicion_meteo_label', 'condicion_firme_label', 'visib_restringida_por_label', 'carretera', 'muertos_30d', 'heridos_graves_30d', 'heridos_leves_30d', 'victimas_30d', 'total_vehiculos', 'peatones_muertos_30d', 'motos_muertos_30d', 'bicis_muertos_30d', 'turismos_muertos_30d', 'vmp_muertos_30d', 'camiones', 'furgonetas', 'autobuses', 'turismos', 'motocicletas', 'vehiculos_totales', 'vehiculos por 1 000\nhabitantes*', 'pct_camiones', 'pct_furgonetas', 'pct_autobuses', 'pct_turismos', 'pct_motocicletas']

Información detallada del dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101306 entries, 0 to 101305
Data columns (total

In [3]:
# Análisis de valores nulos

nulos = df.isnull().sum()
if nulos.any():
    print("\nColumnas con valores nulos:")
    display(nulos[nulos > 0].sort_values(ascending=False))
else:
    print("\nNo se encontraron valores nulos en el dataset")

# Resumen estadístico de variables numéricas clave
vars_numericas = ['muertos_30d', 'heridos_graves_30d', 'heridos_leves_30d', 
                  'victimas_30d', 'vehiculos_totales']
print("\nEstadísticas básicas de variables clave:")
display(df[vars_numericas].describe().round(4))



No se encontraron valores nulos en el dataset

Estadísticas básicas de variables clave:


,muertos_30d,heridos_graves_30d,heridos_leves_30d,victimas_30d,vehiculos_totales
count,101306.0000,101306.0000,101306.0000,101306.0000,1.013060e+05
mean,0.0178,0.0915,1.2266,1.3359,2.030110e+06
std,0.1434,0.3219,0.8516,0.8277,1.798086e+06
min,0.0000,0.0000,0.0000,1.0000,6.336700e+04
25%,0.0000,0.0000,1.0000,1.0000,6.505900e+05
50%,0.0000,0.0000,1.0000,1.0000,1.179625e+06
75%,0.0000,0.0000,1.0000,1.0000,3.699208e+06
max,4.0000,7.0000,42.0000,48.0000,5.444019e+06


- Tenemos 101,306 accidentes registrados en 2023. 
- No tenemos ningún valor nulo
- La mayoría de accidentes tienen 1 víctima
- El promedio de heridos leves es de 1.22

### 1.2 Distribución Temporal de Accidentes

Analizaremos los patrones temporales de los accidentes, incluyendo:
- Distribución semanal
- Patrones mensuales
- Análisis por tramos horarios
- Identificación de períodos críticos

In [4]:
# Análisis mensual
orden_meses = ['ene', 'feb', 'mar', 'abr', 'may', 'jun', 
               'jul', 'ago', 'sep', 'oct', 'nov', 'dic']

# Agregación mensual incluyendo accidentes totales
mensual = df.groupby('mes_label').agg({
    'muertos_30d': ['count', 'sum'],  
    'heridos_graves_30d': 'sum',
    'heridos_leves_30d': 'sum',
    'victimas_30d': 'sum'
}).reindex(orden_meses)

# Aplanar columnas multinivel
mensual.columns = ['total_accidentes', 'fallecidos', 'heridos_graves', 'heridos_leves', 'total_victimas']

# Crear figura con subplots
fig = make_subplots(rows=3, cols=1,
                    subplot_titles=('Accidentes Totales por Mes',
                                  'Víctimas Graves y Fallecidos por Mes',
                                  'Heridos Leves por Mes'),
                    vertical_spacing=0.12,
                    row_heights=[0.33, 0.33, 0.33])

# 1. Gráfico de accidentes totales con etiquetas encima de cada barra
fig.add_trace(
    go.Bar(
        x=mensual.index, 
        y=mensual['total_accidentes'],
        name='Total Accidentes',
        marker_color='lightgrey',
        text=mensual['total_accidentes'],
        textposition='outside',
        texttemplate='%{text:.0f}'
    ),
    row=1, col=1
)

fig.update_traces(cliponaxis=False, selector=dict(type='bar'))

ymax = float(mensual['total_accidentes'].max())
fig.update_yaxes(range=[0, ymax * 1.15], row=1, col=1)

# 2. Gráfico de víctimas graves y fallecidos
for col, nombre, color in zip(['fallecidos', 'heridos_graves'], 
                            ['Fallecidos', 'Heridos Graves'],
                            ['red', 'orange']):
    fig.add_trace(
        go.Scatter(x=mensual.index, 
                  y=mensual[col],
                  name=nombre,
                  mode='lines+markers',
                  line=dict(color=color)),
        row=2, col=1
    )

# 3. Gráfico de heridos leves
fig.add_trace(
    go.Scatter(x=mensual.index, 
              y=mensual['heridos_leves'],
              name='Heridos Leves',
              mode='lines+markers',
              line=dict(color='blue')),
    row=3, col=1
)

# Actualizar layout
fig.update_layout(
    height=1000,
    title_text="Análisis Mensual de Accidentes y Víctimas",
    showlegend=True
)

# Actualizar títulos de ejes
for i in range(1, 4):
    fig.update_xaxes(title_text='Mes', row=i, col=1)
    fig.update_yaxes(title_text='Número', row=i, col=1)

fig.show()

# Calcular proporciones mensuales de severidad
prop_mensual = mensual[['fallecidos', 'heridos_graves', 'heridos_leves']].div(mensual['total_accidentes'], axis=0)

cols_map = {
    'fallecidos': ('Fallecidos', 'red', mensual['fallecidos']),
    'heridos_graves': ('Heridos Graves', 'orange', mensual['heridos_graves']),
    'heridos_leves': ('Heridos Leves', 'blue', mensual['heridos_leves'])
}

fig_prop = go.Figure()

# Añadir trazas apiladas con texto de porcentaje y customdata con conteos absolutos
for col in ['fallecidos', 'heridos_graves', 'heridos_leves']:
    nombre, color = cols_map[col][0], cols_map[col][1]
    cuentas = cols_map[col][2].reindex(orden_meses)  # asegurar orden
    porcentajes = (prop_mensual[col].reindex(orden_meses) * 100).fillna(0)

    fig_prop.add_trace(
        go.Bar(
            name=nombre,
            x=orden_meses,
            y=porcentajes,
            marker_color=color,
            text=[f"{v:.1f}%" if v >= 1.0 else "" for v in porcentajes],  
            textposition='inside',
            textfont=dict(color='white', size=11),
            customdata=cuentas.values,
            hovertemplate="%{x}<br>" + nombre + ": %{y:.1f}% (%{customdata:.0f} personas)<extra></extra>"
        )
    )

fig_prop.update_layout(
    barmode='stack',
    title='Distribución Porcentual de la Severidad por Mes',
    yaxis=dict(range=[0,105], ticksuffix='%', title='Porcentaje'),
    xaxis=dict(title='Mes', categoryorder='array', categoryarray=orden_meses),
    height=520,
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1),
    margin=dict(t=80, b=40)
)

fig_prop.show()

# Estadísticas mensuales detalladas
print("\nEstadísticas mensuales:")
print("\nResumen de accidentes y víctimas por mes:")
stats_display = mensual.copy()
stats_display.columns = ['Total Accidentes', 'Fallecidos', 'Heridos Graves', 
                        'Heridos Leves', 'Total Víctimas']
display(stats_display.round(2))

# Análisis de meses críticos
print("\nMeses críticos:")
print(f"Mes con más accidentes: {mensual['total_accidentes'].idxmax()} ({mensual['total_accidentes'].max():.0f} accidentes)")
print(f"Mes con más víctimas totales: {mensual['total_victimas'].idxmax()} ({mensual['total_victimas'].max():.0f} víctimas)")
print(f"Mes con más fallecidos: {mensual['fallecidos'].idxmax()} ({mensual['fallecidos'].max():.0f} fallecidos)")
print(f"Mes con más heridos graves: {mensual['heridos_graves'].idxmax()} ({mensual['heridos_graves'].max():.0f} heridos graves)")
print(f"Mes con más heridos leves: {mensual['heridos_leves'].idxmax()} ({mensual['heridos_leves'].max():.0f} heridos leves)")

# Calcular tasas de severidad
print("\nTasas de severidad por cada 100 accidentes:")
tasas = pd.DataFrame({
    'Tasa Mortalidad': (mensual['fallecidos'] / mensual['total_accidentes'] * 100).round(2),
    'Tasa Heridos Graves': (mensual['heridos_graves'] / mensual['total_accidentes'] * 100).round(2),
    'Tasa Heridos Leves': (mensual['heridos_leves'] / mensual['total_accidentes'] * 100).round(2)
})
display(tasas)


Estadísticas mensuales:

Resumen de accidentes y víctimas por mes:


,Total Accidentes,Fallecidos,Heridos Graves,Heridos Leves,Total Víctimas
mes_label,,,,,
ene,7964,131,648,9787,10566
feb,7115,118,668,8693,9479
mar,8508,131,765,10165,11061
abr,8656,158,751,10647,11556
may,8804,134,776,10788,11698
jun,8840,151,832,10666,11649
jul,9238,168,907,11604,12679
ago,8197,183,859,10155,11197
sep,8322,167,801,10170,11138



Meses críticos:
Mes con más accidentes: jul (9238 accidentes)
Mes con más víctimas totales: jul (12679 víctimas)
Mes con más fallecidos: ago (183 fallecidos)
Mes con más heridos graves: jul (907 heridos graves)
Mes con más heridos leves: jul (11604 heridos leves)

Tasas de severidad por cada 100 accidentes:


,Tasa Mortalidad,Tasa Heridos Graves,Tasa Heridos Leves
mes_label,,,
ene,1.64,8.14,122.89
feb,1.66,9.39,122.18
mar,1.54,8.99,119.48
abr,1.83,8.68,123.00
may,1.52,8.81,122.54
jun,1.71,9.41,120.66
jul,1.82,9.82,125.61
ago,2.23,10.48,123.89
sep,2.01,9.63,122.21


Estacionalidad:

- Julio y agosto son los meses más críticos en términos de accidentalidad
- Julio registró el mayor número total de víctimas (12,679) y heridos graves (907)
- Agosto tuvo el mayor número de fallecidos (183)

Patrones de severidad:

- La proporción de accidentes graves y mortales es relativamente constante a lo largo del año
- Los meses de verano (julio y agosto) muestran un incremento significativo en todas las categorías de víctimas

Tendencia anual:

- Se observa un claro patrón estacional con picos en verano
- Los meses de invierno tienden a tener menos accidentes en general

#### Análisis por Día de la Semana y Tramo Horario

Analizaremos los patrones de accidentalidad según el día de la semana y el momento del día para identificar períodos de mayor riesgo.

In [5]:
# Preparar datos
orden_dias = ['lunes', 'martes', 'miercoles', 'jueves', 'viernes', 'sabado', 'domingo']
orden_tramos = ['madrugada', 'manana', 'mediodia', 'tarde', 'noche']

# Agregación por día y tramo
temporal = df.groupby(['dia_semana_label', 'tramo_horario_label']).agg({
    'muertos_30d': 'sum',
    'heridos_graves_30d': 'sum',
    'heridos_leves_30d': 'sum',
    'victimas_30d': 'count'  
}).reset_index()

# Crear pivot tables para los heatmaps
pivot_accidentes = pd.pivot_table(
    temporal,
    values='victimas_30d',
    index='dia_semana_label',
    columns='tramo_horario_label',
    aggfunc='sum'
).reindex(orden_dias)[orden_tramos]

# Crear figura con subplots
fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=('Número de Accidentes por Día y Tramo Horario',
                   'Accidentes Totales por Día de la Semana',
                   'Víctimas por Tipo y Día de la Semana'),
    vertical_spacing=0.12,
    row_heights=[0.4, 0.3, 0.3]
)

# 1. Heatmap de accidentes
fig.add_trace(
    go.Heatmap(
        z=pivot_accidentes.values,
        x=orden_tramos,
        y=orden_dias,
        colorscale='YlOrRd',
        text=pivot_accidentes.values.round(0),
        texttemplate="%{text}",
        textfont={"size": 10},
        colorbar=dict(
            title="Número de<br>Accidentes",
            y=0.85,
            len=0.35
        )
    ),
    row=1, col=1
)

# 2. Gráfico de barras para accidentes totales por día
accidentes_por_dia = temporal.groupby('dia_semana_label')['victimas_30d'].sum().reindex(orden_dias)
fig.add_trace(
    go.Bar(
        x=orden_dias,
        y=accidentes_por_dia,
        name='Total Accidentes',
        marker_color='lightgrey',
        text=accidentes_por_dia.round(0),
        textposition='outside',
        textfont=dict(size=12)
    ),
    row=2, col=1
)

# 3. Gráfico de barras para víctimas 
daily_totals = temporal.groupby('dia_semana_label').agg({
    'muertos_30d': 'sum',
    'heridos_graves_30d': 'sum',
    'heridos_leves_30d': 'sum'
}).reindex(orden_dias)

for col, nombre, color, pos in zip(
    ['muertos_30d', 'heridos_graves_30d', 'heridos_leves_30d'],
    ['Fallecidos', 'Heridos Graves', 'Heridos Leves'],
    ['red', 'orange', 'blue'],
    [-0.25, 0, 0.25]
):
    fig.add_trace(
        go.Bar(
            x=orden_dias,
            y=daily_totals[col],
            name=nombre,
            marker_color=color,
            width=0.2,
            offset=pos,
            text=daily_totals[col].round(0),
            textposition='outside',
            textfont=dict(size=10)
        ),
        row=3, col=1
    )

# Actualizar layout
fig.update_layout(
    height=1200,
    showlegend=True,
    title_text='Análisis de Accidentes por Día y Tramo Horario',
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.15,
        xanchor="center",
        x=0.5
    )
)

# Actualizar títulos de ejes y rangos
fig.update_xaxes(title_text='Tramo Horario', row=1, col=1)
fig.update_yaxes(title_text='Día de la Semana', row=1, col=1)

fig.update_xaxes(title_text='Día de la Semana', row=2, col=1)
fig.update_yaxes(title_text='Número de Accidentes', 
                 row=2, col=1,
                 range=[0, accidentes_por_dia.max() * 1.15])  

fig.update_xaxes(title_text='Día de la Semana', row=3, col=1)
fig.update_yaxes(title_text='Número de Víctimas', row=3, col=1)

# Ajustar márgenes
fig.update_layout(margin=dict(t=100, b=150))

# Mostrar la figura
fig.show()

# Mostrar estadísticas por día de la semana
print("\nEstadísticas por día de la semana:")
stats_display = daily_totals.copy()
stats_display['total_accidentes'] = accidentes_por_dia
stats_display = stats_display[['total_accidentes', 'muertos_30d', 'heridos_graves_30d', 'heridos_leves_30d']]
stats_display.columns = ['Total Accidentes', 'Fallecidos', 'Heridos Graves', 'Heridos Leves']
display(stats_display)

# Identificar períodos más críticos
print("\nPeríodos más críticos (Top 10 combinaciones día-tramo):")
top_periodos = temporal.sort_values('victimas_30d', ascending=False).head(10)[
    ['dia_semana_label', 'tramo_horario_label', 'victimas_30d', 
     'muertos_30d', 'heridos_graves_30d', 'heridos_leves_30d']
]
top_periodos.columns = ['Día', 'Tramo', 'Total Accidentes', 'Fallecidos', 'Heridos Graves', 'Heridos Leves']
display(top_periodos.round(2))


Estadísticas por día de la semana:


,Total Accidentes,Fallecidos,Heridos Graves,Heridos Leves
dia_semana_label,,,,
lunes,14890,257,1297,18252
martes,14879,219,1198,17959
miercoles,14966,227,1189,17983
jueves,15096,226,1294,18125
viernes,16420,290,1480,19965
sabado,13353,324,1468,16874
domingo,11702,263,1339,15108



Períodos más críticos (Top 10 combinaciones día-tramo):


,Día,Tramo,Total Accidentes,Fallecidos,Heridos Graves,Heridos Leves
34,viernes,tarde,5275,73,482,6730
9,jueves,tarde,4883,63,407,6037
19,martes,tarde,4827,57,363,6018
14,lunes,tarde,4730,81,401,5793
24,miercoles,tarde,4698,63,347,5818
11,lunes,manana,4422,64,373,5450
16,martes,manana,4408,70,335,5208
21,miercoles,manana,4350,62,344,5054
6,jueves,manana,4144,55,358,4851
31,viernes,manana,4124,78,363,4802


Patrones por día de la semana:

- Los sábados tienen el mayor número de fallecidos (324)
- Los viernes registran el mayor número de heridos graves (1,480) y heridos leves (19,965)
- Los fines de semana muestran una tendencia diferente: más accidentes graves pero menos leves

Distribución horaria:

- El heatmap muestra claramente que las horas de la tarde son las más críticas en general
- La madrugada tiene menos accidentes pero suelen ser más graves
- Los patrones son diferentes entre días laborables y fines de semana

Combinaciones críticas:

- Las tardes de viernes son especialmente problemáticas
- Los sábados por la noche y madrugada del domingo muestran alta severidad
- Los días laborables muestran picos en horas punta (mañana y tarde)

### 1.3 Distribución Geográfica Básica

Analizaremos la distribución espacial de los accidentes, incluyendo:
- Ranking de provincias por número de accidentes
- Tasas de accidentalidad por provincia
- Distribución por tipo de zona
- Análisis por tipo de vía

In [6]:
# Agregación base
provincial = df.groupby('provincia_label').agg(
    muertos_30d=('muertos_30d','sum'),
    heridos_graves_30d=('heridos_graves_30d','sum'),
    heridos_leves_30d=('heridos_leves_30d','sum'),
    vehiculos_totales=('vehiculos_totales','first'),
    victimas_30d=('victimas_30d','count')
).reset_index()
provincial['tasa_mortalidad'] = provincial['muertos_30d'] / provincial['vehiculos_totales'] * 100000
provincial['tasa_accidentalidad'] = provincial['victimas_30d'] / provincial['vehiculos_totales'] * 100000
provincial['total_victimas'] = provincial['muertos_30d'] + provincial['heridos_graves_30d'] + provincial['heridos_leves_30d']

# Detectar columna "vehículos por 1000 habitantes"
vehiculos_por_1000_col = None
for col in df.columns:
    cl = col.lower()
    if 'vehicul' in cl and 'habit' in cl:
        vehiculos_por_1000_col = col
        break
if vehiculos_por_1000_col is None:
    raise ValueError("No se encontró la columna de vehículos por 1000 habitantes.")

vehx1000 = (
    df.groupby('provincia_label')[vehiculos_por_1000_col]
      .first()
      .reset_index()
      .rename(columns={vehiculos_por_1000_col:'vehx1000'})
)
provincial = provincial.merge(vehx1000, on='provincia_label', how='left')
provincial['vehx1000'] = provincial['vehx1000'].replace(0, np.nan)
provincial['tasa_accidentes_por_parque_proxy'] = provincial['victimas_30d'] / provincial['vehx1000']
prov_valid = provincial.dropna(subset=['tasa_accidentes_por_parque_proxy']).copy()

# Selecciones
top15_total = provincial.nlargest(15, 'total_victimas')
top15_tasa  = provincial.nlargest(15, 'tasa_mortalidad')
top10_rate  = prov_valid.nlargest(10, 'tasa_accidentes_por_parque_proxy').sort_values('tasa_accidentes_por_parque_proxy', ascending=False)
bottom10_rate = prov_valid.nsmallest(10, 'tasa_accidentes_por_parque_proxy').sort_values('tasa_accidentes_por_parque_proxy', ascending=True)

# Figura (última fila más grande + menos espacio entre filas)
fig = make_subplots(
    rows=3, cols=2,
    specs=[[{'colspan':2}, None],
           [{'colspan':2}, None],
           [{}, {}]],
    row_heights=[0.32, 0.26, 0.42],            
    subplot_titles=(
        'Top 15 Provincias por Número Total de Víctimas',
        'Top 15 Provincias por Tasa de Mortalidad (por 100,000 vehículos)',
        f'Top 10: Accidentes / ({vehiculos_por_1000_col})',
        f'Bottom 10: Accidentes / ({vehiculos_por_1000_col})'
    ),
    vertical_spacing=0.15                      
)

# --- 1) Top 15 por número total de víctimas (stack) ---
for tipo, color in zip(['muertos_30d', 'heridos_graves_30d', 'heridos_leves_30d'], ['red', 'orange', 'blue']):
    fig.add_trace(
        go.Bar(
            x=top15_total['provincia_label'],
            y=top15_total[tipo],
            name=tipo.replace('_30d', '').replace('_', ' ').title(),
            text=top15_total[tipo].round(0),
            textposition='inside',
            marker_color=color
        ),
        row=1, col=1
    )

# Padding del eje Y para que no se corte el texto del gráfico 1
fig.update_yaxes(range=[0, top15_total['total_victimas'].max() * 1.2], row=1, col=1)


# Fila 2 (tasa mortalidad)
fig.add_trace(
    go.Bar(x=top15_tasa['provincia_label'], y=top15_tasa['tasa_mortalidad'],
           name='Tasa de Mortalidad', text=top15_tasa['tasa_mortalidad'].round(2),
           textposition='outside', marker_color='red'),
    row=2, col=1
)
max_tasa = top15_tasa['tasa_mortalidad'].max()
fig.update_yaxes(range=[0, (max_tasa if max_tasa>0 else 1)*1.15], row=2, col=1)

# Fila 3 (tasa por vehículos/1000 hab.) en horizontal
fig.add_trace(
    go.Bar(x=top10_rate['tasa_accidentes_por_parque_proxy'][::-1],
           y=top10_rate['provincia_label'][::-1],
           orientation='h',
           text=top10_rate['tasa_accidentes_por_parque_proxy'][::-1].round(3),
           textposition='outside',
           name='Top 10 tasa'),
    row=3, col=1
)
fig.add_trace(
    go.Bar(x=bottom10_rate['tasa_accidentes_por_parque_proxy'][::-1],
           y=bottom10_rate['provincia_label'][::-1],
           orientation='h',
           text=bottom10_rate['tasa_accidentes_por_parque_proxy'][::-1].round(3),
           textposition='outside',
           name='Bottom 10 tasa'),
    row=3, col=2
)
pad_left  = (top10_rate['tasa_accidentes_por_parque_proxy'].max() or 1)*1.15
pad_right = (bottom10_rate['tasa_accidentes_por_parque_proxy'].max() or 1)*1.15
fig.update_xaxes(range=[0, pad_left],  row=3, col=1)
fig.update_xaxes(range=[0, pad_right], row=3, col=2)

# Layout
fig.update_layout(
    barmode='stack',
    showlegend=True,
    title_text='Análisis Provincial de Accidentes y Víctimas (normalizado por vehículos/1000 habitantes)',
    height=1500,
    margin=dict(t=90, b=110)  
)

# Ejes y legibilidad
fig.update_xaxes(title_text='Provincia', tickangle=45, automargin=True, row=1, col=1)
fig.update_yaxes(title_text='Número de Víctimas', automargin=True, row=1, col=1)

fig.update_xaxes(title_text='Provincia', tickangle=45, automargin=True, row=2, col=1)
fig.update_yaxes(title_text='Tasa por 100,000 vehículos', automargin=True, row=2, col=1)

fig.update_xaxes(title_text=f"Accidentes / ({vehiculos_por_1000_col})", automargin=True, row=3, col=1)
fig.update_yaxes(title_text="Provincia", automargin=True, row=3, col=1)
fig.update_xaxes(title_text=f"Accidentes / ({vehiculos_por_1000_col})", automargin=True, row=3, col=2)
fig.update_yaxes(title_text="Provincia", automargin=True, row=3, col=2)


fig.show()

# Tablas (15 filas)
print("\nTop 15 provincias por número total de víctimas:")
display(top15_total[['provincia_label','total_victimas','muertos_30d','heridos_graves_30d','heridos_leves_30d','tasa_accidentalidad']].head(15).round(2))

print("\nTop 15 provincias por tasa de mortalidad (por 100,000 vehículos):")
display(top15_tasa[['provincia_label','tasa_mortalidad','muertos_30d','vehiculos_totales','tasa_accidentalidad']].head(15).round(2))

print(f"\nTop 15 por Accidentes / ({vehiculos_por_1000_col}):")
display(prov_valid.nlargest(15,'tasa_accidentes_por_parque_proxy')[['provincia_label','victimas_30d','vehx1000','tasa_accidentes_por_parque_proxy']].round(3))

print(f"\nBottom 15 por Accidentes / ({vehiculos_por_1000_col}):")
display(prov_valid.nsmallest(15,'tasa_accidentes_por_parque_proxy')[['provincia_label','victimas_30d','vehx1000','tasa_accidentes_por_parque_proxy']].round(3))



Top 15 provincias por número total de víctimas:


,provincia_label,total_victimas,muertos_30d,heridos_graves_30d,heridos_leves_30d,tasa_accidentalidad
7,Barcelona,22930,148,1073,21709,478.59
29,Madrid,18911,137,893,17881,269.65
32,Málaga,6789,70,320,6399,378.07
47,Valencia/València,6601,73,399,6129,284.30
42,Sevilla,5608,60,315,5233,285.97
1,Alicante/Alacant,5147,60,404,4683,262.88
31,Murcia,4688,62,259,4367,293.65
17,Cádiz,3935,42,258,3635,335.29
6,"Balears, Illes",3816,64,324,3428,260.24
36,"Palmas, Las",3056,36,127,2893,232.01



Top 15 provincias por tasa de mortalidad (por 100,000 vehículos):


,provincia_label,tasa_mortalidad,muertos_30d,vehiculos_totales,tasa_accidentalidad
43,Soria,21.82,17,77912.0,184.82
26,León,11.51,44,382145.0,142.62
15,Cuenca,11.28,21,186249.0,141.75
24,Huesca,10.88,21,192967.0,194.85
51,Ávila,10.76,15,139405.0,215.92
34,Ourense,10.19,27,265074.0,153.54
41,Segovia,9.46,13,137486.0,234.21
49,Zamora,9.44,14,148307.0,132.16
9,Burgos,9.34,26,278426.0,224.48
27,Lleida,9.23,34,368200.0,382.67



Top 15 por Accidentes / (vehiculos por 1 000
habitantes*):


,provincia_label,victimas_30d,vehx1000,tasa_accidentes_por_parque_proxy
7,Barcelona,17704,637.0,27.793
29,Madrid,14680,794.0,18.489
47,Valencia/València,5237,694.0,7.546
32,Málaga,5006,756.0,6.622
42,Sevilla,3864,690.0,5.600
1,Alicante/Alacant,3887,758.0,5.128
31,Murcia,3464,760.0,4.558
17,Cádiz,2874,685.0,4.196
8,Bizkaia,1965,615.0,3.195
6,"Balears, Illes",2871,921.0,3.117



Bottom 15 por Accidentes / (vehiculos por 1 000
habitantes*):


,provincia_label,victimas_30d,vehx1000,tasa_accidentes_por_parque_proxy
43,Soria,144,871.0,0.165
49,Zamora,196,892.0,0.220
45,Teruel,231,916.0,0.252
35,Palencia,212,837.0,0.253
15,Cuenca,264,945.0,0.279
51,Ávila,301,880.0,0.342
41,Segovia,322,886.0,0.363
28,Lugo,379,916.0,0.414
12,Ceuta,322,763.0,0.422
24,Huesca,376,850.0,0.442


- Madrid, Barcelona y Valencia concentran el mayor número de víctimas totales, pero NO tienen las tasas de mortalidad más altas.

- El 90-95% de las víctimas en todas las provincias son heridos leves (barras azules).

- Paradoja interesante: provincias pequeñas tienen menos accidentes totales pero tasas de mortalidad más altas por vehículo.

- Los accidentes son más frecuentes en zonas urbanas (visible en el gráfico de barras), pero los accidentes en carretera tienden a ser más graves.

- Las provincias con menos accidentes por vehículo son principalmente las del norte de España y las islas.

In [7]:
# Análisis por tipo de zona y vía
zona_via = df.groupby(['zona_label', 'tipo_via_label']).agg({
    'muertos_30d': ['sum', 'count'], 
    'heridos_graves_30d': 'sum',
    'heridos_leves_30d': 'sum',
    'victimas_30d': 'count'
}).reset_index()

# Renombrar las columnas después de la agregación
zona_via.columns = ['zona_label', 'tipo_via_label', 'muertos_30d', 'num_accidentes', 
                   'heridos_graves_30d', 'heridos_leves_30d', 'victimas_30d']

# Calcular totales e índices
zona_via['total_victimas'] = (zona_via['muertos_30d'] + 
                             zona_via['heridos_graves_30d'] + 
                             zona_via['heridos_leves_30d'])
zona_via['indice_gravedad'] = (zona_via['muertos_30d'] + zona_via['heridos_graves_30d']) / zona_via['victimas_30d']
zona_via['victimas_por_accidente'] = zona_via['total_victimas'] / zona_via['num_accidentes']

# Crear visualización con 4 subplots
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'Distribución de Víctimas por Tipo de Zona',
        'Índice de Gravedad por Tipo de Vía',
        'Distribución de Accidentes por Tipo de Zona',
        'Total de Accidentes por Tipo de Vía' 
    ),
    specs=[[{"type": "pie"}, {"type": "bar"}],
           [{"type": "pie"}, {"type": "bar"}]],
    vertical_spacing=0.3,
    horizontal_spacing=0.1
)

# Gráfico 1: Tarta de distribución de víctimas por zona
zona_totals_victimas = zona_via.groupby('zona_label')['total_victimas'].sum()
fig.add_trace(
    go.Pie(labels=zona_totals_victimas.index,
           values=zona_totals_victimas.values,
           name="Víctimas por Zona"),
    row=1, col=1
)

# Gráfico 2: Barras de índice de gravedad por tipo de vía
via_gravedad = zona_via.groupby('tipo_via_label')['indice_gravedad'].mean().sort_values(ascending=False)
fig.add_trace(
    go.Bar(x=via_gravedad.index,
           y=via_gravedad.values,
           name="Índice Gravedad"),
    row=1, col=2
)

# Gráfico 3: Tarta de distribución de accidentes por zona
zona_totals_accidentes = zona_via.groupby('zona_label')['num_accidentes'].sum()
fig.add_trace(
    go.Pie(labels=zona_totals_accidentes.index,
           values=zona_totals_accidentes.values,
           name="Accidentes por Zona"),
    row=2, col=1
)

# Gráfico 4: Barras de total de accidentes por tipo de vía (modificado)
via_accidentes = zona_via.groupby('tipo_via_label')['num_accidentes'].sum().sort_values(ascending=False)
fig.add_trace(
    go.Bar(x=via_accidentes.index,
           y=via_accidentes.values,
           name="Total Accidentes"),
    row=2, col=2
)

# Actualizar layout
fig.update_layout(
    height=1000,
    width=1200,
    title_text="Análisis por Tipo de Zona y Vía - Víctimas y Accidentes",
    showlegend=True,
    margin=dict(t=100, b=100)
)

# Actualizar ejes X para los gráficos de barras
fig.update_xaxes(tickangle=45, row=1, col=2)
fig.update_xaxes(tickangle=45, row=2, col=2)

# Actualizar títulos de ejes para los gráficos de barras
fig.update_yaxes(title_text="Índice de Gravedad", row=1, col=2)
fig.update_yaxes(title_text="Número de Accidentes", row=2, col=2)  

fig.show()

# Mostrar estadísticas detalladas
print("\nEstadísticas por tipo de zona y vía (ordenado por número de víctimas):")
display(zona_via[['zona_label', 'tipo_via_label', 'total_victimas', 'num_accidentes', 
                 'victimas_por_accidente', 'indice_gravedad']]
        .sort_values('total_victimas', ascending=False)
        .head(10)
        .round(2))

print("\nEstadísticas por tipo de zona y vía (ordenado por número de accidentes):")
display(zona_via[['zona_label', 'tipo_via_label', 'num_accidentes', 'total_victimas', 
                 'victimas_por_accidente', 'indice_gravedad']]
        .sort_values('num_accidentes', ascending=False)
        .head(10)
        .round(2))


Estadísticas por tipo de zona y vía (ordenado por número de víctimas):


,zona_label,tipo_via_label,total_victimas,num_accidentes,victimas_por_accidente,indice_gravedad
2,Calle,Calle,77702,62726,1.24,0.08
12,Carretera,Carretera Convencional de calzada única,27670,18742,1.48,0.20
10,Carretera,Autovía,12588,7727,1.63,0.12
8,Carretera,Autopista de peaje,3361,2097,1.60,0.10
9,Carretera,Autopista libre,2683,1663,1.61,0.11
14,Carretera,Otro,2540,1830,1.39,0.11
13,Carretera,Carretera Convencional de doble calzada,2284,1646,1.39,0.11
3,Calle,Otro,1679,1294,1.30,0.15
24,Travesía,Carretera Convencional de calzada única,1349,972,1.39,0.15
11,Carretera,Camino vecinal,1317,947,1.39,0.23



Estadísticas por tipo de zona y vía (ordenado por número de accidentes):


,zona_label,tipo_via_label,num_accidentes,total_victimas,victimas_por_accidente,indice_gravedad
2,Calle,Calle,62726,77702,1.24,0.08
12,Carretera,Carretera Convencional de calzada única,18742,27670,1.48,0.20
10,Carretera,Autovía,7727,12588,1.63,0.12
8,Carretera,Autopista de peaje,2097,3361,1.60,0.10
14,Carretera,Otro,1830,2540,1.39,0.11
9,Carretera,Autopista libre,1663,2683,1.61,0.11
13,Carretera,Carretera Convencional de doble calzada,1646,2284,1.39,0.11
3,Calle,Otro,1294,1679,1.30,0.15
24,Travesía,Carretera Convencional de calzada única,972,1349,1.39,0.15
11,Carretera,Camino vecinal,947,1317,1.39,0.23


Zona Urbana vs. Interurbana:

- La zona urbana concentra la mayoría de víctimas (aprox. 60%)
- También concentra la mayoría de accidentes (gráfico de tarta inferior)

Gravedad por tipo de vía:

- Las carreteras convencionales muestran el índice de gravedad más alto
- Las autovías, a pesar de la velocidad, tienen menor índice de gravedad

Accidentes por tipo de vía:

- Vías urbanas tienen el mayor número de accidentes
- Seguidas por carreteras convencionales
- Autovías y autopistas tienen menos accidentes totales

Correlación interesante:

- Zonas urbanas: Más accidentes pero menos graves
- Zonas interurbanas: Menos accidentes pero más graves
- Las vías de servicio y ramales de enlace muestran los índices más bajos tanto en accidentes como en gravedad

## 2. Análisis de Severidad y Víctimas por Tipo de Usuario

Analizaremos la distribución de víctimas según el tipo de usuario de la vía:
- Peatones
- Motociclistas
- Ciclistas
- Usuarios de turismos
- Usuarios de VMP (Vehículos de Movilidad Personal)

In [8]:
# Análisis por tipo de usuario
cols_usuarios = ['peatones_muertos_30d', 'motos_muertos_30d', 
                'bicis_muertos_30d', 'turismos_muertos_30d', 'vmp_muertos_30d']

usuarios_total = pd.DataFrame({
    'Tipo': [col.replace('_muertos_30d', '').replace('_', ' ').title() 
             for col in cols_usuarios],
    'Fallecidos': [df[col].sum() for col in cols_usuarios]
})

# Calcular porcentajes
usuarios_total['Porcentaje'] = usuarios_total['Fallecidos'] / usuarios_total['Fallecidos'].sum() * 100

# Análisis temporal por tipo de usuario
usuarios_mensual = df.groupby('mes_label')[cols_usuarios].sum().reindex(
    ['ene', 'feb', 'mar', 'abr', 'may', 'jun', 
     'jul', 'ago', 'sep', 'oct', 'nov', 'dic']
)

# Crear visualización
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Distribución de Fallecidos por Tipo de Usuario',
                   'Proporción de Fallecidos (%)',
                   'Evolución Mensual por Tipo de Usuario',
                   ''),
    specs=[[{"type": "bar"}, {"type": "pie"}],
           [{"type": "scatter", "colspan": 2}, None]],  
    vertical_spacing=0.2,  
    row_heights=[0.4, 0.6]  
)

# 1. Gráfico de barras de cantidades absolutas
fig.add_trace(
    go.Bar(x=usuarios_total['Tipo'],
           y=usuarios_total['Fallecidos'],
           name='Fallecidos',
           text=usuarios_total['Fallecidos'],
           textposition='outside'),
    row=1, col=1
)

# 2. Gráfico circular de porcentajes
fig.add_trace(
    go.Pie(labels=usuarios_total['Tipo'],
           values=usuarios_total['Porcentaje'],
           name='Porcentaje'),
    row=1, col=2
)

# 3. Evolución mensual (ocupando toda la fila)
for col, nombre in zip(cols_usuarios, usuarios_total['Tipo']):
    fig.add_trace(
        go.Scatter(x=usuarios_mensual.index,
                  y=usuarios_mensual[col],
                  name=nombre,
                  mode='lines+markers+text',  
                  text=usuarios_mensual[col],  
                  textposition='top center'),  
        row=2, col=1
    )

# Actualizar layout
fig.update_layout(
    height=1000, 
    title_text="Análisis por Tipo de Usuario",
    showlegend=True,
    barmode='group'
)

# Ajustar ejes
fig.update_xaxes(tickangle=45, row=2, col=1)
fig.update_xaxes(title_text="Mes", row=2, col=1)
fig.update_yaxes(title_text="Número de Fallecidos", row=2, col=1)

fig.show()

# Mostrar estadísticas
print("\nEstadísticas por tipo de usuario:")
display(usuarios_total.sort_values('Fallecidos', ascending=False).round(2))

# Calcular ratios relevantes
print("\nRatios relevantes:")
total_fallecidos = usuarios_total['Fallecidos'].sum()
print(f"- Proporción de motociclistas: {(usuarios_total.loc[1, 'Fallecidos']/total_fallecidos*100):.2f}%")
print(f"- Proporción de peatones: {(usuarios_total.loc[0, 'Fallecidos']/total_fallecidos*100):.2f}%")
print(f"- Ratio motos/turismos: {(usuarios_total.loc[1, 'Fallecidos']/usuarios_total.loc[3, 'Fallecidos']):.2f}")


Estadísticas por tipo de usuario:


,Tipo,Fallecidos,Porcentaje
3,Turismos,703,43.64
1,Motos,455,28.24
0,Peatones,353,21.91
2,Bicis,90,5.59
4,Vmp,10,0.62



Ratios relevantes:
- Proporción de motociclistas: 28.24%
- Proporción de peatones: 21.91%
- Ratio motos/turismos: 0.65


Distribución de Fallecidos:

- Los ocupantes de turismos son las principales víctimas (43.64%, 703 fallecidos)
- Seguido por motociclistas (28.24%, 455 fallecidos)
- Los peatones representan una proporción significativa (21.91%, 353 fallecidos)
- Ciclistas y VMP (Vehículos de Movilidad Personal) tienen menor incidencia (5.59% y 0.62% respectivamente)

Tendencias Temporales:

- Se observa un claro patrón estacional con picos en los meses de verano
- Las motocicletas muestran la mayor variación estacional, con máximos en meses cálidos
- Los accidentes de peatones tienden a ser más constantes durante el año

Ratios Significativos:

- Por cada fallecido en turismo hay 0.65 fallecidos en moto, lo que indica un riesgo desproporcionadamente alto para los motociclistas dado su menor número en circulación
- Los usuarios vulnerables (peatones + ciclistas + motociclistas) suman más del 55% de las víctimas mortales

Implicaciones para la Seguridad Vial:

- Necesidad de medidas específicas para proteger a usuarios vulnerables
- Importancia de campañas estacionales, especialmente en verano para motociclistas
- Los VMP, aunque con números bajos actualmente, merecen atención preventiva dado su creciente uso

Vulnerabilidad Relativa:

- A pesar de que los turismos son el medio más común, los motociclistas y peatones muestran una sobrerrepresentación en las estadísticas de mortalidad
- Los ciclistas, aunque con menor número absoluto, muestran una vulnerabilidad significativa (90 fallecidos)

## 3. Factores Contextuales

### 3.1 Condiciones Ambientales

Analizaremos la influencia de diversos factores ambientales en los accidentes:
- Condiciones de iluminación
- Condiciones meteorológicas
- Estado del firme
- Factores de visibilidad

In [9]:
# 1. Análisis de iluminación (día vs noche)
iluminacion = df.groupby('condicion_iluminacion_label').agg({
    'muertos_30d': ['sum', 'size']  
}).reset_index()

# Renombrar columnas
iluminacion.columns = ['condicion_iluminacion_label', 'fallecidos', 'num_accidentes']
iluminacion['victimas_por_accidente'] = iluminacion['fallecidos'] / iluminacion['num_accidentes']

# 2. Análisis de condiciones meteorológicas
meteo = df['condicion_meteo_label'].value_counts().reset_index()
meteo.columns = ['condicion', 'accidentes']

# Agrupar condiciones meteorológicas en categorías más amplias
meteo['condicion'] = meteo['condicion'].replace({
    'Buen tiempo': 'Buen tiempo',
    'Lluvia débil': 'Lluvia',
    'Lluvia fuerte': 'Lluvia',
    'Niebla intensa': 'Visibilidad Reducida',
    'Niebla ligera': 'Visibilidad Reducida',
    'Nieve': 'Otras',
    'Granizo': 'Otras',
    'Otro': 'Otras'
})
meteo = meteo.groupby('condicion')['accidentes'].sum().reset_index()

# Crear visualización con specs apropiados para gráfico circular
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=(
        'Comparativa de Accidentes: Día vs Noche',
        'Distribución de Accidentes por Condiciones Meteorológicas'
    ),
    specs=[[{"secondary_y": True}],
           [{"type": "domain"}]],  
    vertical_spacing=0.2,
    row_heights=[0.5, 0.5]
)

# 1. Gráfico de barras comparativo día/noche
fig.add_trace(
    go.Bar(
        x=iluminacion['condicion_iluminacion_label'],
        y=iluminacion['num_accidentes'],
        text=iluminacion['num_accidentes'],
        textposition='auto',
        name='Número de Accidentes'
    ),
    row=1, col=1
)

# Añadir línea de víctimas por accidente
fig.add_trace(
    go.Scatter(
        x=iluminacion['condicion_iluminacion_label'],
        y=iluminacion['victimas_por_accidente'],
        mode='lines+markers+text',
        text=iluminacion['victimas_por_accidente'].round(3),
        textposition='top center',
        name='Fallecidos por Accidente',
        yaxis='y2'
    ),
    row=1, col=1,
    secondary_y=True
)

# 2. Gráfico de pie para condiciones meteorológicas
fig.add_trace(
    go.Pie(
        labels=meteo['condicion'],
        values=meteo['accidentes'],
        textposition='inside',
        textinfo='label+percent',
        hole=0.4
    ),
    row=2, col=1
)

# Actualizar layout
fig.update_layout(
    height=900,
    title_text="Análisis de Condiciones Ambientales en Accidentes",
    showlegend=True
)

# Actualizar títulos de ejes
fig.update_yaxes(title_text="Número de Accidentes", secondary_y=False, row=1, col=1)
fig.update_yaxes(title_text="Fallecidos por Accidente", secondary_y=True, row=1, col=1)

# Ajustar márgenes y espaciado
fig.update_layout(
    margin=dict(t=100, b=50, l=50, r=50)
)

fig.show()

# Mostrar estadísticas clave
print("\nEstadísticas de accidentes por condición de iluminación:")
display(iluminacion.round(3))

print("\nDistribución de accidentes por condiciones meteorológicas:")
display(meteo.sort_values('accidentes', ascending=False))


Estadísticas de accidentes por condición de iluminación:


,condicion_iluminacion_label,fallecidos,num_accidentes,victimas_por_accidente
0,"Amanecer o atardecer, con luz artificial",29,2722,0.011
1,"Amanecer o atardecer, sin luz artificial",95,3494,0.027
2,"Luz del día natural, solar",1081,72623,0.015
3,Sin especificar,0,31,0.000
4,Sin luz natural ni artificial,358,5620,0.064
5,Sin luz natural y con iluminación artificial e...,198,14177,0.014
6,Sin luz natural y con iluminación artificial n...,45,2639,0.017



Distribución de accidentes por condiciones meteorológicas:


,condicion,accidentes
0,Despejado,87730
2,Lluvia,5963
4,Nublado,5378
5,Se desconoce,2079
1,Granizando,57
3,Nevando,57
6,Sin especificar,42


Las estadísticas muestran algunos datos interesantes:

Iluminación:

- La mayoría de los accidentes (72,623) ocurren con luz del día natural
- La tasa más alta de fallecidos por accidente (0.064) ocurre en condiciones sin luz natural ni artificial
- Las condiciones de amanecer/atardecer sin luz artificial también muestran una tasa relativamente alta de fallecidos (0.027)

Condiciones meteorológicas:

- La gran mayoría de los accidentes (87,730) ocurren en condiciones despejadas
- La lluvia es la segunda condición más común con 5,963 accidentes
- Las condiciones extremas como granizo y nieve son muy poco frecuentes (57 accidentes cada una)

### 3.2 Características de la Vía

Analizaremos las características específicas de las vías donde ocurren los accidentes:
- Titularidad de la vía
- Tipo de vía
- Características específicas
- Patrones de concentración de accidentes

In [10]:
# Análisis de características de la vía
via_cols = ['titularidad_via_label', 'tipo_via_label']

# Análisis por características de la vía
caracteristicas_via = df.groupby(via_cols).agg({
    'muertos_30d': ['sum', 'size'],  # sum para fallecidos, size para contar accidentes
    'heridos_graves_30d': 'sum',
    'heridos_leves_30d': 'sum'
}).reset_index()

# Renombrar columnas
caracteristicas_via.columns = ['titularidad_via_label', 'tipo_via_label', 'muertos_30d', 
                             'num_accidentes', 'heridos_graves_30d', 'heridos_leves_30d']

# Calcular métricas
caracteristicas_via['total_victimas'] = (caracteristicas_via['muertos_30d'] + 
                                       caracteristicas_via['heridos_graves_30d'] + 
                                       caracteristicas_via['heridos_leves_30d'])
caracteristicas_via['indice_gravedad'] = (caracteristicas_via['muertos_30d'] + 
                                         caracteristicas_via['heridos_graves_30d']) / caracteristicas_via['num_accidentes']

# Crear visualización
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=(
        'Distribución de Víctimas por Titularidad de la Vía',
        'Top 10 Combinaciones con Mayor Número de Víctimas'
    ),
    specs=[[{"type": "pie"}],
           [{"type": "bar"}]],
    vertical_spacing=0.3,
    row_heights=[0.4, 0.6]
)

# 1. Gráfico circular por titularidad
titularidad_total = caracteristicas_via.groupby('titularidad_via_label')['total_victimas'].sum()
fig.add_trace(
    go.Pie(
        labels=titularidad_total.index,
        values=titularidad_total.values,
        textinfo='label+percent',
        hole=0.4
    ),
    row=1, col=1
)

# 2. Top 10 combinaciones con más víctimas
top10_via = caracteristicas_via.nlargest(10, 'total_victimas')
fig.add_trace(
    go.Bar(
        x=top10_via['total_victimas'],
        y=[f"{row['titularidad_via_label']} - {row['tipo_via_label']}" 
           for _, row in top10_via.iterrows()],
        orientation='h',
        text=top10_via['total_victimas'].round(0).astype(int),
        textposition='auto',
    ),
    row=2, col=1
)

# Actualizar layout
fig.update_layout(
    height=800,
    title_text="Análisis de Características de la Vía",
    showlegend=True
)

# Ajustar el layout de la barra horizontal
fig.update_xaxes(title_text="Número total de víctimas", row=2, col=1)
fig.update_yaxes(title_text="Tipo de vía", row=2, col=1)

fig.show()

# Mostrar estadísticas clave
print("\nTop 10 combinaciones más peligrosas:")
display(caracteristicas_via[['titularidad_via_label', 'tipo_via_label', 
                           'total_victimas', 'indice_gravedad', 'num_accidentes']]
        .sort_values('total_victimas', ascending=False)
        .head(10)
        .round(3))


Top 10 combinaciones más peligrosas:


,titularidad_via_label,tipo_via_label,total_victimas,indice_gravedad,num_accidentes
29,Municipal,Calle,58462,0.083,47103
43,Otra,Calle,17973,0.066,14734
5,Autonómica,Carretera Convencional de calzada única,11236,0.236,7550
16,Estatal,Autovía,9047,0.117,5456
59,"Provincial, Cabildo/Consell",Carretera Convencional de calzada única,7100,0.210,4891
45,Otra,Carretera Convencional de calzada única,5568,0.055,4043
19,Estatal,Carretera Convencional de calzada única,4766,0.240,2980
47,Otra,Otro,2433,0.074,1813
14,Estatal,Autopista de peaje,1996,0.104,1228
2,Autonómica,Autovía,1580,0.213,968


## 4. Análisis de Riesgo por Tipo de Vehículo

### 4.1 Relación entre Composición del Parque y Accidentalidad

- Analizar la composición del parque de vehículos en España
- Comparar con la implicación en accidentes mortales de cada tipo
- Identificar vehículos con riesgo desproporcionado
- Calcular índices de riesgo relativo por categoría de vehículo



In [17]:
# ========================================
# PARTE 1: ANÁLISIS A NIVEL NACIONAL
# ========================================

# Calcular el parque total de España (sumando todas las provincias, evitando duplicados)
parque_espana = df.groupby('provincia_label')[['camiones', 'furgonetas', 'autobuses', 
                                                 'turismos', 'motocicletas']].first().sum()

# Calcular porcentajes del parque nacional
total_vehiculos_espana = parque_espana.sum()
pct_parque_espana = (parque_espana / total_vehiculos_espana * 100)

print("=" * 70)
print("COMPOSICIÓN DEL PARQUE DE VEHÍCULOS EN ESPAÑA (2023)")
print("=" * 70)
print(f"\nTotal de vehículos: {total_vehiculos_espana:,.0f}")
print("\nDistribución por tipo:")
for tipo, valor in parque_espana.items():
    pct = pct_parque_espana[tipo]
    print(f"  {tipo.capitalize():15s}: {valor:>12,.0f} ({pct:>5.2f}%)")

# Calcular implicación en accidentes con fallecidos por tipo de vehículo
accidentes_con_muertos_motos = df[df['motos_muertos_30d'] > 0].shape[0]
accidentes_con_muertos_turismos = df[df['turismos_muertos_30d'] > 0].shape[0]
accidentes_con_muertos_bicis = df[df['bicis_muertos_30d'] > 0].shape[0]
accidentes_con_muertos_peatones = df[df['peatones_muertos_30d'] > 0].shape[0]

# Total de accidentes con fallecidos (solo motos + turismos para comparar con el parque)
total_acc_mortales_vehiculos = accidentes_con_muertos_motos + accidentes_con_muertos_turismos

# Calcular porcentajes RELATIVOS (del total de accidentes mortales con estos vehículos)
pct_acc_mortales_motos = (accidentes_con_muertos_motos / total_acc_mortales_vehiculos * 100)
pct_acc_mortales_turismos = (accidentes_con_muertos_turismos / total_acc_mortales_vehiculos * 100)

print("\n" + "=" * 70)
print("IMPLICACIÓN EN ACCIDENTES CON FALLECIDOS")
print("=" * 70)
print(f"\nTotal de accidentes registrados: {len(df):,.0f}")
print(f"Total de accidentes con fallecidos (motos + turismos): {total_acc_mortales_vehiculos:,.0f}")
print("\nDistribución de accidentes con fallecidos:")
print(f"  Motocicletas:    {accidentes_con_muertos_motos:>6,} ({pct_acc_mortales_motos:>5.2f}%)")
print(f"  Turismos:        {accidentes_con_muertos_turismos:>6,} ({pct_acc_mortales_turismos:>5.2f}%)")
print(f"\nOtros usuarios vulnerables:")
print(f"  Peatones:        {accidentes_con_muertos_peatones:>6,}")
print(f"  Bicicletas:      {accidentes_con_muertos_bicis:>6,}")

# Crear tabla comparativa (solo para motos y turismos que tenemos en el parque)
# Calculamos porcentajes solo de motos y turismos para comparación justa
total_motos_turismos_parque = parque_espana['turismos'] + parque_espana['motocicletas']
pct_parque_turismos = (parque_espana['turismos'] / total_motos_turismos_parque * 100)
pct_parque_motos = (parque_espana['motocicletas'] / total_motos_turismos_parque * 100)

comparacion_nacional = pd.DataFrame({
    'Tipo_Vehiculo': ['Turismos', 'Motocicletas'],
    'Vehiculos_Parque': [parque_espana['turismos'], parque_espana['motocicletas']],
    'Pct_Parque': [pct_parque_turismos, pct_parque_motos],
    'Accidentes_con_Fallecidos': [accidentes_con_muertos_turismos, accidentes_con_muertos_motos],
    'Pct_Acc_Mortales': [pct_acc_mortales_turismos, pct_acc_mortales_motos]
})

# Calcular índice de riesgo
comparacion_nacional['Indice_Riesgo'] = (
    comparacion_nacional['Pct_Acc_Mortales'] / comparacion_nacional['Pct_Parque']
)

print("\n" + "=" * 70)
print("TABLA COMPARATIVA: PARQUE VS ACCIDENTALIDAD MORTAL")
print("=" * 70)
print("(Comparación entre Turismos y Motocicletas)")
display(comparacion_nacional.round(2))

COMPOSICIÓN DEL PARQUE DE VEHÍCULOS EN ESPAÑA (2023)

Total de vehículos: 34,734,313

Distribución por tipo:
  Camiones       :    2,466,998 ( 7.10%)
  Furgonetas     :    2,681,233 ( 7.72%)
  Autobuses      :       66,638 ( 0.19%)
  Turismos       :   25,356,594 (73.00%)
  Motocicletas   :    4,162,850 (11.98%)

IMPLICACIÓN EN ACCIDENTES CON FALLECIDOS

Total de accidentes registrados: 101,306
Total de accidentes con fallecidos (motos + turismos): 1,060

Distribución de accidentes con fallecidos:
  Motocicletas:       446 (42.08%)
  Turismos:           614 (57.92%)

Otros usuarios vulnerables:
  Peatones:           342
  Bicicletas:          88

TABLA COMPARATIVA: PARQUE VS ACCIDENTALIDAD MORTAL
(Comparación entre Turismos y Motocicletas)


,Tipo_Vehiculo,Vehiculos_Parque,Pct_Parque,Accidentes_con_Fallecidos,Pct_Acc_Mortales,Indice_Riesgo
0,Turismos,25356594.0,85.9,614,57.92,0.67
1,Motocicletas,4162850.0,14.1,446,42.08,2.98


In [23]:
# ========================================
# VISUALIZACIONES - ANÁLISIS NACIONAL
# ========================================

# Crear figura con 2 subplots
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=(
        'Comparación:<br>% del Parque vs % Accidentes Mortales',
        'Índice de Riesgo Relativo<br>por Tipo de Vehículo'
    ),
    specs=[[{"type": "bar"}, {"type": "bar"}]],
    horizontal_spacing=0.2
)

# Gráfico 1: Barras agrupadas comparativas
x_labels = comparacion_nacional['Tipo_Vehiculo']

fig.add_trace(
    go.Bar(
        name='% del Parque',
        x=x_labels,
        y=comparacion_nacional['Pct_Parque'],
        text=comparacion_nacional['Pct_Parque'].round(1),
        texttemplate='%{text}%',
        textposition='outside',
        marker_color='lightblue'
    ),
    row=1, col=1
)

fig.add_trace(
    go.Bar(
        name='% Acc. Mortales',
        x=x_labels,
        y=comparacion_nacional['Pct_Acc_Mortales'],
        text=comparacion_nacional['Pct_Acc_Mortales'].round(1),
        texttemplate='%{text}%',
        textposition='outside',
        marker_color='red'
    ),
    row=1, col=1
)

# Gráfico 2: Índice de riesgo con línea de referencia
colores = ['green' if x < 1 else 'red' for x in comparacion_nacional['Indice_Riesgo']]

fig.add_trace(
    go.Bar(
        x=x_labels,
        y=comparacion_nacional['Indice_Riesgo'],
        text=comparacion_nacional['Indice_Riesgo'].round(2),
        texttemplate='%{text}x',
        textposition='outside',
        marker_color=colores,
        showlegend=False
    ),
    row=1, col=2
)

# Añadir línea de referencia en 1.0
fig.add_hline(
    y=1.0, 
    line_dash="dash", 
    line_color="black", 
    annotation_text="Riesgo Proporcional (1.0)",
    annotation_position="right",
    row=1, col=2
)

# Actualizar layout
fig.update_layout(
    height=600,
    title_text="Análisis de Riesgo por Tipo de Vehículo - España 2023",
    showlegend=True,
    barmode='group',
    margin=dict(t=160, b=50, l=50, r=50)  # Aumentado el margen superior
)

# Actualizar ejes
fig.update_yaxes(title_text="Porcentaje (%)", row=1, col=1, range=[0, max(comparacion_nacional['Pct_Parque'].max(), comparacion_nacional['Pct_Acc_Mortales'].max()) * 1.15])
fig.update_yaxes(title_text="Índice de Riesgo", row=1, col=2, range=[0, comparacion_nacional['Indice_Riesgo'].max() * 1.2])
fig.update_xaxes(title_text="Tipo de Vehículo", row=1, col=1)
fig.update_xaxes(title_text="Tipo de Vehículo", row=1, col=2)

fig.show()


1. **Las motocicletas tienen casi 3 veces más riesgo relativo** que los turismos:
   - Representan solo el 14.1% del parque de vehículos
   - Pero están implicadas en el 42.08% de los accidentes con fallecidos
   - Índice de riesgo: **2.98x** (casi el triple de lo esperado)

2. **Los turismos son relativamente más seguros:**
   - Representan el 85.9% del parque
   - Solo están implicados en el 57.92% de los accidentes mortales
   - Índice de riesgo: **0.67x** (1.5 veces menos peligrosos de lo esperado)

3. **Implicaciones:** 
   - Las motocicletas son vehículos de **alto riesgo desproporcionado**
   - Se necesitan políticas específicas de seguridad para motociclistas
   - El bajo índice de turismos puede deberse a mayor seguridad pasiva (airbags, estructura, etc.)

### 4.2 Análisis Complementario: Composición del Parque Vehicular por Provincia

- Distribución de los diferentes tipos de vehículos en las provincias de España.
- Relación con muertes

In [ ]:
# ========================================
# COMPOSICIÓN DEL PARQUE POR PROVINCIA
# ========================================

# Obtener datos únicos por provincia 
parque_provincial = df.groupby('provincia_label')[
    ['pct_turismos', 'pct_motocicletas', 'pct_camiones', 'pct_furgonetas', 'pct_autobuses', 
     'vehiculos_totales', 'muertos_30d']
].agg({
    'pct_turismos': 'first',
    'pct_motocicletas': 'first',
    'pct_camiones': 'first',
    'pct_furgonetas': 'first',
    'pct_autobuses': 'first',
    'vehiculos_totales': 'first',
    'muertos_30d': 'sum'  
}).reset_index()

# Seleccionar Top 15 provincias por número total de vehículos
top15_provincias = parque_provincial.nlargest(15, 'vehiculos_totales').sort_values('vehiculos_totales', ascending=True)

# Crear gráfico de barras apiladas horizontales
fig = go.Figure()

# Añadir cada tipo de vehículo como una barra apilada
tipos_vehiculos = [
    ('Turismos', 'pct_turismos', '#1f77b4'),
    ('Motocicletas', 'pct_motocicletas', '#ff7f0e'),
    ('Camiones', 'pct_camiones', '#2ca02c'),
    ('Furgonetas', 'pct_furgonetas', '#d62728'),
    ('Autobuses', 'pct_autobuses', '#9467bd')
]

for nombre, columna, color in tipos_vehiculos:
    fig.add_trace(go.Bar(
        name=nombre,
        y=top15_provincias['provincia_label'],
        x=top15_provincias[columna],
        orientation='h',
        marker_color=color,
        text=top15_provincias[columna].round(1),
        texttemplate='%{text}%',
        textposition='inside',
        hovertemplate='<b>%{y}</b><br>' + nombre + ': %{x:.1f}%<extra></extra>'
    ))

fig.update_layout(
    title='Composición del Parque de Vehículos en las 15 Provincias Principales',
    xaxis_title='Porcentaje del Parque (%)',
    yaxis_title='Provincia',
    barmode='stack',
    height=700,
    showlegend=True,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
    xaxis=dict(range=[0, 100])
)

fig.show()

# Mostrar estadísticas clave
print("=" * 80)
print("COMPOSICIÓN DEL PARQUE - TOP 15 PROVINCIAS")
print("=" * 80)
print("\nProvincias ordenadas por tamaño de parque vehicular:")
display(top15_provincias[['provincia_label', 'vehiculos_totales', 'pct_turismos', 
                          'pct_motocicletas', 'pct_camiones', 'muertos_30d']].round(2))

# Encontrar provincias con más % de motos
print("\n" + "=" * 80)
print("PROVINCIAS CON MAYOR PRESENCIA DE MOTOCICLETAS")
print("=" * 80)
top_motos = parque_provincial.nlargest(10, 'pct_motocicletas')[
    ['provincia_label', 'pct_motocicletas', 'muertos_30d']
]
print("\nTop 10 provincias con mayor % de motocicletas en el parque:")
display(top_motos.round(2))

# Correlación simple entre % motos y fallecidos
print("\n" + "=" * 80)
print("INSIGHT CLAVE")
print("=" * 80)
correlacion = parque_provincial[['pct_motocicletas', 'muertos_30d']].corr().iloc[0, 1]
print(f"\nCorrelación entre % de motocicletas y fallecidos por provincia: {correlacion:.3f}")
if correlacion > 0.3:
    print("→ Hay una correlación POSITIVA moderada: más motos = más fallecidos")
elif correlacion < -0.3:
    print("→ Hay una correlación NEGATIVA: más motos = menos fallecidos (inesperado)")
else:
    print("→ La correlación es DÉBIL: otros factores influyen más que solo el % de motos")

COMPOSICIÓN DEL PARQUE - TOP 15 PROVINCIAS

Provincias ordenadas por tamaño de parque vehicular:


,provincia_label,vehiculos_totales,pct_turismos,pct_motocicletas,pct_camiones,muertos_30d
4,Asturias,718710.0,73.33,9.58,5.32,49
21,Granada,744522.0,66.33,14.96,7.24,35
37,Pontevedra,751263.0,73.06,11.59,5.86,43
14,"Coruña, A",835637.0,77.24,7.86,5.55,38
17,Cádiz,857180.0,70.98,15.08,6.04,42
40,Santa Cruz de Tenerife,902954.0,66.64,9.73,12.42,33
36,"Palmas, Las",943514.0,67.40,9.19,12.29,36
6,"Balears, Illes",1103193.0,67.66,15.96,7.83,64
31,Murcia,1179625.0,70.27,11.38,7.17,62
32,Málaga,1324079.0,68.22,15.33,6.30,70



PROVINCIAS CON MAYOR PRESENCIA DE MOTOCICLETAS

Top 10 provincias con mayor % de motocicletas en el parque:


,provincia_label,pct_motocicletas,muertos_30d
12,Ceuta,23.74,1
7,Barcelona,19.28,148
20,Girona,16.17,51
6,"Balears, Illes",15.96,64
32,Málaga,15.33,70
17,Cádiz,15.08,42
21,Granada,14.96,35
19,Gipuzkoa,14.27,33
44,Tarragona,13.51,55
30,Melilla,13.40,0



INSIGHT CLAVE

Correlación entre % de motocicletas y fallecidos por provincia: 0.339
→ Hay una correlación POSITIVA moderada: más motos = más fallecidos


---

## 5. Conclusiones del Análisis Exploratorio

Este análisis exploratorio de los **101,306 accidentes** registrados en España durante 2023 ha permitido identificar patrones claros de accidentalidad y factores de riesgo que merecen especial atención desde el punto de vista de la seguridad vial.

### 5.1 Patrones Temporales: El Verano como Período Crítico

El análisis temporal revela una **fuerte estacionalidad** en la accidentalidad vial española. **Julio y agosto** concentran los picos máximos tanto en número de accidentes como en severidad: julio registró 12,679 víctimas totales y agosto alcanzó las 183 víctimas mortales, cifras que superan en más de un 30% los meses de menor incidencia. 

A nivel semanal, los **viernes y sábados** emergen como los días más problemáticos, siendo especialmente preocupantes las **tardes de viernes** (horas de salida laboral y comienzo de fin de semana) y las **madrugadas de sábado a domingo**, períodos donde la combinación de mayor tráfico, ocio nocturno y posible consumo de alcohol incrementa significativamente el riesgo.

### 5.2 Geografía de la Accidentalidad: Volumen vs. Gravedad

La distribución geográfica muestra una dicotomía interesante. Mientras que **Madrid, Barcelona y Valencia** lideran en número absoluto de accidentes (lógico por su densidad poblacional y tráfico), las **provincias más pequeñas** presentan tasas de mortalidad por vehículo significativamente más altas. Este patrón sugiere que las carreteras interurbanas y secundarias, más presentes en provincias menos urbanizadas, son escenarios de accidentes de mayor severidad.

En cuanto al tipo de vía, existe un claro **trade-off entre frecuencia y gravedad**: las zonas urbanas concentran el 60% de los accidentes pero con menor letalidad, mientras que las **carreteras convencionales interurbanas** muestran el índice de gravedad más alto, confirmando que a mayor velocidad y menor infraestructura de seguridad, mayor es la probabilidad de desenlace fatal.

### 5.3 Usuarios Vulnerables: La Desproporcionada Mortalidad de Motociclistas

El análisis por tipo de usuario revela uno de los hallazgos más críticos de este estudio: **los usuarios vulnerables (motociclistas, peatones y ciclistas) representan más del 55% de las víctimas mortales**, a pesar de ser minoría en el tráfico total.

Especialmente preocupante es el caso de las **motocicletas**, que muestran un **índice de riesgo de 2.98x** respecto a lo esperado por su presencia en el parque vehicular. Esto significa que, aunque solo representan el 14.1% del parque de vehículos en España, están implicadas en el 42% de los accidentes con fallecidos. Por el contrario, los **turismos** (85.9% del parque) solo participan en el 57.9% de accidentes mortales, mostrando un índice de riesgo de 0.67x (más seguros de lo esperado), probablemente debido a mejores sistemas de seguridad pasiva (airbags, estructuras reforzadas, ABS, etc.).

El análisis provincial complementa este hallazgo: existe una **correlación positiva moderada (r=0.371)** entre el porcentaje de motocicletas en el parque y el número de accidentes totales, y una correlación similar (r=0.339) con la mortalidad. Provincias como **Barcelona** (19.3% de motos, 17,704 accidentes, 148 fallecidos) y **Ceuta** (23.7% de motos, la mayor proporción del país) ejemplifican esta relación, aunque factores adicionales como densidad de tráfico y características de las vías también influyen.

### 5.4 Factores Contextuales: Más Allá de las Condiciones Adversas

Contrariamente a lo que podría esperarse, la mayoría de los accidentes (87%) ocurren en **condiciones meteorológicas favorables** y con **luz natural**. Sin embargo, el análisis de severidad muestra que los accidentes en condiciones de **oscuridad sin iluminación artificial** presentan una tasa de fallecidos por accidente significativamente más alta (0.064 vs. promedio general), lo que sugiere que aunque son menos frecuentes, son más letales.

Este patrón refuerza la idea de que la accidentalidad no depende tanto de condiciones extraordinarias, sino de **factores estructurales y comportamientos rutinarios**: exceso de velocidad, distracciones, no uso de sistemas de retención, y la propia configuración de las infraestructuras.

### 5.5 Recomendaciones Estratégicas para Políticas de Seguridad Vial

Basándonos en los hallazgos de este EDA, se proponen las siguientes líneas de acción prioritarias:

1. **Campañas específicas para motociclistas**: Dada la desproporcionada mortalidad (índice 2.98x), es urgente reforzar la formación, el uso obligatorio de equipamiento de protección certificado (casco homologado, chaquetas con protecciones, guantes) y la concienciación sobre maniobras de riesgo. Las campañas deberían intensificarse en **verano**, período de mayor uso recreativo de motocicletas.

2. **Intervenciones preventivas estacionales y semanales**: Incrementar controles de velocidad y alcohol/drogas durante los **viernes y sábados por la tarde-noche**, especialmente en **julio y agosto**. Considerar restricciones de velocidad dinámicas en períodos de alta densidad de tráfico.

3. **Mejora de infraestructuras en carreteras convencionales**: Priorizar inversiones en señalización, barreras de seguridad (especialmente quitamiedos más efectivos para motoristas), mejora del firme y eliminación de puntos negros en las **carreteras convencionales interurbanas**, que muestran el mayor índice de gravedad.

4. **Iluminación en zonas de alto riesgo**: Aunque la mayoría de accidentes ocurren de día, la alta letalidad en condiciones de oscuridad sin luz artificial justifica inversiones en iluminación de travesías, cruces y tramos peligrosos.

5. **Enfoque provincial diferenciado**: Las provincias con altos porcentajes de motocicletas en el parque (Barcelona, Ceuta, Girona, Baleares, Málaga) deberían desarrollar planes locales específicos de seguridad motociclista, considerando sus particularidades geográficas y de movilidad.

6. **Protección de peatones y ciclistas**: Con 353 peatones y 90 ciclistas fallecidos, es necesario seguir invirtiendo en infraestructuras de movilidad activa segura (aceras más anchas, carriles bici protegidos, pasos de peatones bien señalizados) y en educación vial para todos los usuarios.

### 5.6 Reflexión Final

Este análisis confirma que la seguridad vial es un **problema multifactorial** donde confluyen diseño de infraestructuras, comportamientos humanos, tecnología vehicular y políticas públicas. Los datos demuestran que **no existe una única solución mágica**, pero sí áreas de intervención prioritarias donde los esfuerzos pueden tener un impacto significativo.

El caso de las motocicletas es paradigmático: representan libertad y movilidad ágil, pero conllevan un riesgo desproporcionado que no puede ignorarse. Reducir esta brecha requerirá un esfuerzo conjunto de administraciones, fabricantes (mejorando sistemas de seguridad activa como ABS y control de tracción), escuelas de conducción (formación más rigurosa) y, sobre todo, **concienciación individual** de cada motorista sobre su vulnerabilidad.

Los 1,611 fallecidos y más de 100,000 accidentes registrados en 2023 no son solo estadísticas: representan familias rotas, vidas truncadas y un enorme coste social y económico. Este EDA pretende aportar evidencia empírica para que las decisiones de política pública estén fundamentadas en datos, priorizando las intervenciones con mayor potencial de impacto. **Cada vida salvada justifica el esfuerzo.**